# ✨ UnQCreator Engine Launcher

Welcome to the UnQCreator Engine Launcher! This notebook sets up your cloud-based engine backend for creating amazing videos.

## 🚀 Quick Start:
1. Click the "Run" button below to start the engine
2. Wait for the setup to complete (5-10 minutes)
3. Your public URL will appear automatically when ready
4. Use the copy button to easily copy the URL

## 🖥️ Hardware Recommendations:
For best performance, select a T4 or A100 GPU runtime (Runtime → Change runtime type → Hardware accelerator → GPU).

In [ ]:
#@title 🚀 Launch UnQCreator Engine
#@markdown Click the play button to start the UnQCreator Engine.

import os
import time
import asyncio
import re
from subprocess import Popen, PIPE
from IPython.display import HTML, display, Javascript

# Hide output for cleaner experience
import warnings
warnings.filterwarnings('ignore')

# Function to create a clean progress indicator without showing code output
def show_progress(message):
    # Use HTML for cleaner display
    display(HTML(f"""
    <div style="padding: 10px; margin-bottom: 10px; border-left: 5px solid #0366d6; background-color: #f0f7ff;">
        <p style="margin: 0; color: #0366d6; font-weight: 500;">{message}</p>
    </div>
    """))

# Suppress standard output for cleaner experience
import sys
from io import StringIO

class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = StringIO()
        return sys.stdout
        
    def __exit__(self, *args):
        sys.stdout = self._original_stdout

# Step 1: Clone the repository
show_progress("📥 Setting up the UnQCreator Engine...")

with SuppressOutput():
    !git clone -q https://github.com/Sandeepgaddam5432/unqcreator.git /content/unqcreator
    %cd /content/unqcreator

# Step 2: Install dependencies with correct path to requirements.txt
show_progress("📦 Installing dependencies... (This may take a few minutes)")

with SuppressOutput():
    !pip install -q -r video_genarater/UnQ-YT-Auto/requirements.txt

# Step 3: Setup Cloudflare Quick Tunnel
show_progress("🔒 Setting up secure connection...")

with SuppressOutput():
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
    !chmod +x /usr/local/bin/cloudflared

# Step 4: Launch Quick Tunnel and capture the URL
show_progress("🚇 Establishing secure tunnel...")

# Create a log file to capture the tunnel output
with SuppressOutput():
    !touch /tmp/cloudflared.log

# Start the tunnel in the background and redirect output to the log file
tunnel_process = Popen(
    ['/usr/local/bin/cloudflared', 'tunnel', '--url', 'http://localhost:8188'],
    stdout=PIPE, stderr=PIPE, universal_newlines=True
)

# Function to extract the tunnel URL from cloudflared output
def get_tunnel_url():
    # Wait for the tunnel to initialize
    time.sleep(5)
    
    # Read from the process output
    tunnel_url = None
    max_attempts = 20
    attempt = 0
    
    while tunnel_url is None and attempt < max_attempts:
        attempt += 1
        # Read from stderr as cloudflared logs to stderr
        line = tunnel_process.stderr.readline()
        if not line:
            time.sleep(1)
            continue
            
        # Look for the trycloudflare.com URL in the output
        match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
        if match:
            tunnel_url = match.group(0)
            break
    
    # If we couldn't find the URL in the process output, try a different approach
    if tunnel_url is None:
        # Run cloudflared tunnel info command to get the URL
        with SuppressOutput() as captured:
            info_output = !cloudflared tunnel info
        
        for line in info_output:
            match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
            if match:
                tunnel_url = match.group(0)
                break
    
    return tunnel_url

# Get the tunnel URL
tunnel_url = get_tunnel_url()

if tunnel_url:
    # Create JavaScript function for copy button
    copy_js = f"""
    function copyToClipboard() {{
        const el = document.createElement('textarea');
        el.value = '{tunnel_url}';
        document.body.appendChild(el);
        el.select();
        document.execCommand('copy');
        document.body.removeChild(el);
        
        // Change button text temporarily
        const btn = document.getElementById('copyButton');
        const originalText = btn.innerHTML;
        btn.innerHTML = '✓ Copied!';
        btn.style.backgroundColor = '#10B981';
        setTimeout(() => {{
            btn.innerHTML = originalText;
            btn.style.backgroundColor = '#0366D6';
        }}, 2000);
    }}
    """
    
    # Show pretty note box with the tunnel URL and copy button
    display(HTML(f"""
    <div style="background-color: #f0f7ff; padding: 25px; border-radius: 10px; margin: 20px 0; text-align: center; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);">
      <h2 style="margin-top: 0; color: #0366d6; font-size: 24px;">🎉 Your Engine is Ready!</h2>
      <p style="margin-bottom: 20px; font-size: 16px;">Your UnQCreator Engine is now available at:</p>
      <div style="background-color: #e1effe; border: 1px solid #bfdbfe; border-radius: 6px; padding: 15px; margin: 15px auto; max-width: 90%; display: flex; align-items: center; justify-content: space-between;">
        <code style="font-size: 16px; color: #1e40af; word-break: break-all; margin-right: 10px;">{tunnel_url}</code>
        <button id="copyButton" onclick="copyToClipboard()" style="background-color: #0366D6; color: white; border: none; border-radius: 4px; padding: 8px 16px; font-weight: 500; cursor: pointer; white-space: nowrap;">📋 Copy URL</button>
      </div>
      <p style="margin-top: 20px; font-size: 14px;">Use this URL in your frontend Settings page to connect to your engine.</p>
    </div>
    <script>
    {copy_js}
    </script>
    """))
    
    # Display hardware information
    with SuppressOutput() as captured:
        gpu_info = !nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader
    
    if gpu_info and len(gpu_info) > 0:
        gpu_name = gpu_info[0].split(',')[0].strip()
        display(HTML(f"""
        <div style="background-color: #f0f8ff; padding: 15px; border-left: 5px solid #3b82f6; border-radius: 5px; margin: 20px 0;">
          <p style="margin: 0; font-size: 14px;">🖥️ <strong>Hardware:</strong> {gpu_name}</p>
          <p style="margin: 5px 0 0; font-size: 14px; color: #4b5563;">Your engine is running with optimal performance settings for this hardware.</p>
        </div>
        """))
else:
    display(HTML("""
    <div style="background-color: #fee2e2; padding: 15px; border-left: 5px solid #ef4444; border-radius: 5px; margin: 20px 0;">
      <h3 style="margin-top: 0; color: #b91c1c;">❌ Connection Error</h3>
      <p>Could not establish Cloudflare Tunnel. Please try again or check your internet connection.</p>
    </div>
    """))

# Launch the server with the correct path and allow any origin
# This runs in the background and its output is hidden
!cd video_genarater/UnQ-YT-Auto && python main.py --listen --enable-cors-header "*" > /dev/null 2>&1 &

# Keep the notebook running
display(HTML("""
<div style="background-color: #ecfdf5; padding: 15px; border-left: 5px solid #10b981; border-radius: 5px; margin: 20px 0;">
  <p style="margin: 0;">✅ <strong>Engine is running!</strong> Keep this tab open to maintain your connection.</p>
</div>
"""))